## Pipelines de ML usando SparkML


<p style='color: red'>El propósito de este cuaderno es mostrarte cómo usar SparkML para crear pipelines de aprendizaje automático.


### __Tabla de contenidos__

<ol>
  <li>
    <a href="#Objectives">Objetivos
    </a>
  </li>
  <li>
    <a href="#Datasets">Conjuntos de datos
    </a>
  </li>
  <li>
    <a href="#Setup">Configuración
    </a>
    <ol>
      <li>
        <a href="#Installing-Required-Libraries">Instalación de librerías requeridas
        </a>
      </li>
      <li>
        <a href="#Importing-Required-Libraries">Importación de librerías requeridas
        </a>
      </li>
    </ol>
  </li>
  <li>
    <a href="#Examples">Ejemplos
    </a>
    <ol>
    <li>
      <a href="#Task-1---Load-data-set">Tarea 1 - Cargar conjunto de datos
      </a>
    </li>
    <li>
      <a href="#Task-2---Define-pipeline-stages">Tarea 2 - Definir etapas del pipeline
      </a>
    </li>
    <li>
      <a href="#Task-3---Build-the-pipeline">Tarea 3 - Construir el pipeline
      </a>
    </li>
    <li>
      <a href="#Task-4---Split-the-data">Tarea 4 - Dividir los datos
      </a>
    </li>
    <li>
      <a href="#Task-5---Fit-the-pipeline">Tarea 5 - Ajustar el pipeline
      </a>
    </li>
    <li>
      <a href="#Task-6---Evaluate-the-model">Tarea 6 - Evaluar el modelo
      </a>
    </li>
    </ol>
  </li>
  <li>
    <a href="#Exercises">Ejercicios
    </a>
  </li>
  <ol>
    <li>
      <a href="#Exercise-1---Load-data-set">Ejercicio 1 - Cargar conjunto de datos
      </a>
    </li>
    <li>
      <a href="#Exercise-2---Define-pipeline-stages">Ejercicio 2 - Definir etapas del pipeline
      </a>
    </li>
    <li>
      <a href="#Exercise-3---Build-the-pipeline">Ejercicio 3 - Construir el pipeline
      </a>
    </li>
    <li>
      <a href="#Exercise-4---Split-the-data">Ejercicio 4 - Dividir los datos
      </a>
    </li>
    <li>
      <a href="#Exercise-5---Fit-the-pipeline">Ejercicio 5 - Ajustar el pipeline
      </a>
    </li>
    <li>
      <a href="#Exercise-6---Evaluate-the-model">Ejercicio 6 - Evaluar el modelo
      </a>
    </li>
  </ol>
</ol>



### Objetivos

Después de completar este cuaderno podrás:

 - Crear un pipeline de aprendizaje automático.
 - Agregar etapas al pipeline.
 - Ejecutar el pipeline.
 - Crear un pipeline de aprendizaje automático para regresión.
 - Crear un pipeline de aprendizaje automático para clasificación.


### Conjuntos de datos

En este cuaderno utilizaremos los siguientes conjuntos de datos:

 - Versión modificada del conjunto de datos de millaje de automóviles. Conjunto de datos original disponible en https://archive.ics.uci.edu/ml/datasets/auto+mpg 
 - Versión modificada del conjunto de datos Iris. Conjunto de datos original disponible en https://archive.ics.uci.edu/ml/datasets/Iris 


#### Configuración


In [ ]:
#!pip install pyspark==3.1.2 -q
#!pip install findspark -q

#### Importación de librerías requeridas

_Recomendamos importar todas las librerías necesarias en un solo lugar (aquí):_


In [ ]:
# Puedes usar esta sección para suprimir advertencias generadas por tu código
def warn(*args, **kwargs):
    pass

import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifica el proceso de uso de Apache Spark con Python
import findspark
findspark.init()

# Importar funciones y clases para modelos de Machine Learning en SparkML
from pyspark.ml.regression import LinearRegression  # Regresión Lineal
from pyspark.ml.classification import LogisticRegression  # Regresión Logística

# Importar herramientas para la transformación de datos
from pyspark.ml.feature import VectorAssembler  # Ensamblaje de vectores de características
from pyspark.ml.feature import StandardScaler  # Normalización de características
from pyspark.ml.feature import StringIndexer  # Indexación de variables categóricas

# Crear una sesión de Spark
from pyspark.sql import SparkSession

# Importar funciones y clases para la creación de pipelines
from pyspark.ml import Pipeline  

# Importar funciones y clases para la evaluación de modelos
from pyspark.ml.evaluation import RegressionEvaluator  # Evaluador para modelos de regresión
from pyspark.ml.evaluation import MulticlassClassificationEvaluator  # Evaluador para clasificación multicategoría



### Ejemplos


### Tarea 1 - Cargar conjunto de datos


Crear SparkSession


In [ ]:
# Ignoramos warnings de comandos de SparkSession

spark = SparkSession.builder.appName("Ejemplo de un pipeline de ML").getOrCreate()

Descargar el archivo de datos


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/mpg.csv


Carga el conjunto de datos en el DataFrame de Spark


In [ ]:
# Usando la función spark.read.csv, cargamos los datos en un DataFrame.
# El parámetro header=True indica que el archivo CSV tiene una fila de encabezado.
# El parámetro inferSchema=True permite que Spark detecte automáticamente los tipos de datos de las columnas.

# Se carga el conjunto de datos mpg
mpg_data = spark.read.csv("mpg.csv", header=True, inferSchema=True)



Imprime el esquema del conjunto de datos


In [ ]:
mpg_data.printSchema()

Muestra las primeras 5 filas del conjunto de datos


In [ ]:
mpg_data.show(5)

### Tarea 2 - Definimos etapas del pipeline


In [ ]:
# Etapa 1 - Ensambla las columnas de entrada en un único vector
vectorAssembler = VectorAssembler(
    inputCols=["Weight", "Horsepower", "Engine Disp"],  # Columnas de entrada
    outputCol="features"  # Nombre de la columna de salida con el vector ensamblado
)

# Etapa 2 - Escala las características utilizando StandardScaler
scaler = StandardScaler(
    inputCol="features",  # Columnas de entrada a escalar
    outputCol="scaledFeatures"  # Nombre de la columna de salida con las características escaladas
)

# Etapa 3 - Crea una instancia de regresión lineal
lr = LinearRegression(
    featuresCol="scaledFeatures",  # Columnas de características a usar en la regresión
    labelCol="MPG"  # Variable objetivo (consumo de combustible en millas por galón)
)


### Tarea 3 - Se construye el pipeline



In [ ]:
# Construcción del pipeline
# Se mencionan todas las etapas del pipeline en el orden de ejecución.
pipeline = Pipeline(stages=[vectorAssembler, scaler, lr])

### Tarea 4 - Se divide los datos


In [ ]:
# Se divide los datos en conjuntos de entrenamiento y prueba
(trainingData, testData) = mpg_data.randomSplit([0.7, 0.3], seed=42)

### Tarea 5 -  Se ajusta el pipeline


In [ ]:
# Ajusta el pipeline a los datos de entrenamiento
# Ignora cualquier advertencia. Las advertencias se deben a configuraciones simplificadas y ajustes de seguridad del entorno de laboratorio.

model = pipeline.fit(trainingData)


### Tarea 6 - Evaluación del modelo


Realizar predicciones en los datos de prueba


In [ ]:
predictions = model.transform(testData)

Imprime el valor de RMSE


In [ ]:
evaluator = RegressionEvaluator(labelCol="MPG", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) =", rmse)

Detenine la sesión de Spark


In [ ]:
spark.stop()

### Ejercicios


#### Ejercicio 1 - Cargar conjunto de datos


Crea SparkSession con appname "Ejercicio de pipeline ML"


In [ ]:
#Ignore any warnings by SparkSession command

spark = #TODO

<details>
    <summary>Haz clic aquí para una pista</summary>
    
Utiliza SparkSession.builder

</details>


<details>
    <summary>Haz clic aquí para la solución</summary>
    
```python
spark = SparkSession.builder.appName("ML Pipeline Exercise").getOrCreate()
```

</details>


Descarga el conjunto de datos iris


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/iris.csv


Carga el conjunto de datos en el DataFrame de Spark


In [ ]:
iris_data = #TODO

<details>
    <summary>Haz clic aquí para una pista</summary>
    
Utiliza el método spark.read.csv

</details>


<details>
    <summary>Haz clic aquí para la solución</summary>
    
```python
iris_data = spark.read.csv("iris.csv", header=True, inferSchema=True)
```

</details>


Imprime el esquema del conjunto de datos


In [ ]:
iris_data.printSchema()

#### Observa que la "Species" es una columna de tipo cadena


Muestra las primeras 5 filas del conjunto de datos


In [ ]:
iris_data.show(5)

#### Ejercicio 2 - Definir etapas del pipeline


Etapa 1 - Crea una etapa indexadora utilizando StringIndexer que convierta la columna Species en una columna numérica llamada "label"


In [ ]:
indexer = #TODO

<details>
    <summary>Haz clic aquí para una pista</summary>
    
Utiliza StringIndexer con inputCol como "Species" y outputCol como "label"

</details>


<details>
    <summary>Haz clic aquí para la solución</summary>
    
```python
indexer = StringIndexer(inputCol="Species", outputCol="label")
```

</details>


Etapa 2 - Crea una etapa vectorAssembler que genere un vector de características llamado features utilizando "SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"


In [ ]:
vectorAssembler = #TODO

<details>
    <summary>Haz clic aquí para una pista</summary>
    
Utiliza el VectorAssembler

</details>


<details>
    <summary>Haz clic aquí para la solución</summary>
    
```python
vectorAssembler = VectorAssembler(inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm","PetalWidthCm"], outputCol="features")
```

</details>


Etapa 3 - Creaa una etapa escaladora que escale las características utilizando StandardScaler, y nombra la columna de salida como scaledFeatures

In [ ]:
scaler = #TODO

<details>
    <summary>Haz clic aquí para una pista</summary>
    
Utiliza el StandardScaler

</details>


<details>
    <summary>Haz clic aquí para la solución</summary>
    
```python
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
```

</details>


Etapa 4 - Crea una etapa de regresión logística usando featuresCol="scaledFeatures", labelCol="label"


In [ ]:
classifier = #TODO

<details>
    <summary>Haz clic aquí para una pista</summary>
    
Utiliza SparkSession.builder

</details>


<details>
    <summary>Haz clic aquí para la solución</summary>
    
```python
classifier = LogisticRegression(featuresCol="scaledFeatures", labelCol="label")
```

</details>


#### Ejercicio 3 - Construir el pipeline


Construye un pipeline con las cuatro etapas creadas anteriormente.


In [ ]:
pipeline = #TODO

<details>
    <summary>Haz clic aquí para una pista</summary>
    
Construye el pipeline utilizando las 4 etapas creadas anteriormente

</details>


<details>
    <summary>Haz clic aquí para la solución</summary>
    
```python
pipeline = Pipeline(stages=[indexer,vectorAssembler, scaler, classifier])
```

</details>


#### Ejercicio 4 - Dividir los datos


Divide los datos en conjuntos de entrenamiento y prueba


In [ ]:
(trainingData, testData) = iris_data.randomSplit([0.7, 0.3], seed=42)

#### Ejercicio 5 - Ajustar el pipeline


Ajusta el pipeline a los datos de entrenamiento


In [ ]:
model = #TODO


<details>
    <summary>Haz clic aquí para una pista</summary>
    
Utiliza el método fit del pipeline

</details>


<details>
    <summary>Haz clic aquí para la solución</summary>
    
```python
model = pipeline.fit(trainingData)
```

</details>


#### Ejercicio 6 - Evaluar el modelo


Realiza predicciones en los datos de prueba


In [ ]:
predictions = #TODO

<details>
    <summary>Haz clic aquí para una pista</summary>
    
Utiliza el método transform del modelo

</details>


<details>
    <summary>Haz clic aquí para la solución</summary>
    
```python
predictions = model.transform(testData)
```

</details>


### Evaluar el rendimiento del modelo


Imprime el valor de RMSE


In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy =", accuracy)


Detiene la sesión de Spark


In [ ]:
spark.stop()